# Neresbot 0.1

Modelos:
* Convolução
* Long Short-Term Memory
* SARIMA

In [7]:
import pandas as pd
import requests
import json
from pprint import pprint
import yahooquery as yq
import os

# Download dos Dados

Os símbolos dos tickers são obtidos manualmente do site: https://www.dadosdemercado.com.br/bolsa/acoes

In [8]:
tickers = pd.read_csv("acoes-listadas.csv")

In [9]:
tickers

,Código,Nome
0,PETR4,Petrobras
1,MGLU3,Magazine Luiza
2,HAPV3,Hapvida
3,CIEL3,Cielo
4,B3SA3,B3
...,...,...
522,TKNO3,Tekno
523,COCE6,Coelce
524,MGEL3,Mangels
525,CTSA8,Santanense


https://yahooquery.dpguthrie.com/

É criado um dicionário que conterá, como chave, o símbolo do ticker e, como valor, o dataframe relacionando medidas e períodos (dias).

Em cada dataframe é adicionado o devido dia da semana, numericamente.

In [10]:
# Os símbolos das empresas brasileiras recebem o sufixo ".SA".

símbolos = tickers['Código'] + '.SA'

In [13]:
# Verificando símbolos inválidos.

print(yq.Ticker(símbolos, backoff_factor=1, progress=True, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True).invalid_symbols)

None


In [22]:
dados = {}
falhas = []

for ticker in símbolos:
    query = yq.Ticker(ticker, backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
    try:
        histórico = query.history(period='1y', interval='1d',adj_timezone=True)
    except:
        falhas.append(ticker)
        print(ticker,"falhou.")
    else:
        dados[ticker] = histórico.reset_index()

print(len(dados),"tickers salvos com sucesso.")
print(len(falhas),"tickers apresentaram falha")

WIZS3.SA falhou.
LLIS3.SA falhou.
BRML3.SA falhou.
DMMO3.SA falhou.
GETT3.SA falhou.
GETT4.SA falhou.
SULA4.SA falhou.
SULA3.SA falhou.
CEPE5.SA falhou.
CEPE6.SA falhou.
BKBR3.SA falhou.
MTIG4.SA falhou.
BLUT4.SA falhou.
BLUT3.SA falhou.
MODL4.SA falhou.
CARD3.SA falhou.
FNCN3.SA falhou.
LCAM3.SA falhou.
BIDI4.SA falhou.
BIDI3.SA falhou.
EEEL4.SA falhou.
EEEL3.SA falhou.
BBRK3.SA falhou.
CESP6.SA falhou.
CESP3.SA falhou.
CESP5.SA falhou.
MOSI3.SA falhou.
POWE3.SA falhou.
GNDI3.SA falhou.
LAME4.SA falhou.
LAME3.SA falhou.
OMGE3.SA falhou.
IGTA3.SA falhou.
JPSA3.SA falhou.
BRDT3.SA falhou.
JBDU4.SA falhou.
JBDU3.SA falhou.
HGTX3.SA falhou.
CCPR3.SA falhou.
DTEX3.SA falhou.
VVAR3.SA falhou.
PNVL4.SA falhou.
TESA3.SA falhou.
BTOW3.SA falhou.
LINX3.SA falhou.
BTTL3.SA falhou.
GPCP3.SA falhou.
GPCP4.SA falhou.
SMLS3.SA falhou.
MMXM3.SA falhou.
BSEV3.SA falhou.
CNTO3.SA falhou.
TIET4.SA falhou.
TIET3.SA falhou.
CEPE3.SA falhou.
CALI4.SA falhou.
BPAR3.SA falhou.
APTI4.SA falhou.
467 tickers sa

In [31]:
errados = []

for i,j in dados.items():
    if 'date' not in j.columns:
        print(i,j.columns)
        errados.append(i)

for i in errados:
    del dados[i]
        

In [35]:
dados2 = {k: v.set_index('date') for k, v in dados.items()}


In [ ]:
# Retentar

Retentar = False


if Retentar:
    for ticker in falhas:
        print(ticker)
        query = yq.Ticker(ticker, backoff_factor=0.2, progress=True, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
        try:
            histórico = query.history(period='1y', interval='1d',adj_timezone=True)
        except:
            falhas.append(ticker)
        else:
            dados[ticker] = histórico.reset_index().set_index('date')

print(len(dados),"tickers salvos com sucesso.")
print(len(falhas),"tickers apresentaram falha")

WIZS3.SA


  0%|          | 0/1 [00:03<?, ?it/s]


LLIS3.SA


  0%|          | 0/1 [00:03<?, ?it/s]


BRML3.SA


...

  0%|          | 0/1 [00:03<?, ?it/s]


BLUT3.SA


  0%|          | 0/1 [00:03<?, ?it/s]


MODL4.SA


  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
dicionario_dataframes = {...}  # Seu dicionário de dataframes
pasta_destino = 'dados'  # Caminho para a pasta de destino

# Verifica se a pasta de destino existe, se não, cria-a
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

# Itera pelo dicionário e salva cada dataframe como um arquivo CSV
for ticker, df in dados2.items():
    nome_arquivo = f'{ticker}.csv'  # Nome do arquivo CSV
    caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)  # Caminho completo para o arquivo CSV
    
    df.to_csv(caminho_arquivo, index=False)  # Salva o dataframe como arquivo CSV
    
    print(f'Arquivo {nome_arquivo} salvo com sucesso.')



Arquivo PETR4.SA.csv salvo com sucesso.
Arquivo MGLU3.SA.csv salvo com sucesso.
Arquivo HAPV3.SA.csv salvo com sucesso.
Arquivo CIEL3.SA.csv salvo com sucesso.
Arquivo B3SA3.SA.csv salvo com sucesso.
Arquivo VIIA3.SA.csv salvo com sucesso.
Arquivo ITSA4.SA.csv salvo com sucesso.
Arquivo CVCB3.SA.csv salvo com sucesso.
Arquivo VALE3.SA.csv salvo com sucesso.
Arquivo BBDC4.SA.csv salvo com sucesso.
Arquivo ITUB4.SA.csv salvo com sucesso.
Arquivo AMER3.SA.csv salvo com sucesso.
Arquivo ASAI3.SA.csv salvo com sucesso.
Arquivo TRAD3.SA.csv salvo com sucesso.
Arquivo BBAS3.SA.csv salvo com sucesso.
Arquivo OIBR3.SA.csv salvo com sucesso.
Arquivo PETR3.SA.csv salvo com sucesso.
Arquivo ABEV3.SA.csv salvo com sucesso.
Arquivo LREN3.SA.csv salvo com sucesso.
Arquivo CPLE6.SA.csv salvo com sucesso.
Arquivo COGN3.SA.csv salvo com sucesso.
Arquivo GOLL4.SA.csv salvo com sucesso.
Arquivo RAIZ4.SA.csv salvo com sucesso.
Arquivo RAIL3.SA.csv salvo com sucesso.
Arquivo SUZB3.SA.csv salvo com sucesso.


In [42]:
dados3 = {}  # Dicionário vazio para armazenar as tabelas

# Percorre todos os arquivos na pasta
for nome_arquivo in os.listdir('dados'):
    if nome_arquivo.endswith('.csv'):  # Verifica se o arquivo é um arquivo CSV
        caminho_arquivo = os.path.join('dados', nome_arquivo)  # Caminho completo para o arquivo
        
        dados3[os.path.splitext(nome_arquivo)[0]] = pd.read_csv(caminho_arquivo)

# Imprime o dicionário com as tabelas
print(len(dados3))

print(dados['PETR4.SA'])

for i in dados3:
    print(i)

466
       symbol        date       open       high        low      close  \
0    PETR4.SA  2022-06-17  28.170000  28.450001  26.129999  27.309999   
1    PETR4.SA  2022-06-20  26.020000  28.250000  25.910000  27.620001   
2    PETR4.SA  2022-06-21  27.799999  28.160000  26.690001  27.070000   
3    PETR4.SA  2022-06-22  26.580000  27.549999  26.389999  26.990000   
4    PETR4.SA  2022-06-23  26.980000  27.379999  26.120001  26.490000   
..        ...         ...        ...        ...        ...        ...   
245  PETR4.SA  2023-06-12  30.219999  30.990000  30.040001  30.809999   
246  PETR4.SA  2023-06-13  29.240000  29.240000  28.590000  28.860001   
247  PETR4.SA  2023-06-14  28.940001  30.100000  28.879999  30.100000   
248  PETR4.SA  2023-06-15  30.200001  30.850000  29.280001  29.389999   
249  PETR4.SA  2023-06-16  29.230000  29.920000  29.040001  29.639999   

        volume   adjclose  dividends  
0    184606500  16.253057   0.000000  
1     92581800  16.437550   0.000000  
2 

# Convolução

## Colunas para os rótulos

Cada linha analisa se há aumento expressivo nos próximos 5 períodos.

Se não houver, o rótulo será 0; Caso contrário, será o número de dias em que o aumento ocorre.  
Há uma variável ajustável para determinar o nível do aumento.

In [43]:
# Coluna para os rótulos

for i,j in dados3.items():
    j['Rótulos'] = 0

In [ ]:
# Metaparâmetros

Aumento = 0.1

Prospecção = 5

In [ ]:
for key, value in dados3.items():
    for 

In [49]:
for i,row in dados3['PETR4.SA'].iterrows():
    print(row)
    print('_')

symbol        PETR4.SA
open             28.17
high         28.450001
low          26.129999
close        27.309999
volume       184606500
adjclose     16.253057
dividends          0.0
Rótulos              0
Name: 0, dtype: object
_
symbol        PETR4.SA
open             26.02
high             28.25
low              25.91
close        27.620001
volume        92581800
adjclose      16.43755
dividends          0.0
Rótulos              0
Name: 1, dtype: object
_
symbol        PETR4.SA
open         27.799999
high             28.16
low          26.690001
close            27.07
volume        85179500
adjclose     16.110224
dividends          0.0
Rótulos              0
Name: 2, dtype: object
_
symbol        PETR4.SA
open             26.58
high         27.549999
low          26.389999
close            26.99
volume        57874700
adjclose     16.062614
dividends          0.0
Rótulos              0
Name: 3, dtype: object
_
symbol        PETR4.SA
open             26.98
high         27.379999
low

In [44]:
dados3['PETR4.SA'].head()

,symbol,open,high,low,close,volume,adjclose,dividends,Rótulos
0,PETR4.SA,28.170000,28.450001,26.129999,27.309999,184606500,16.253057,0.0,0
1,PETR4.SA,26.020000,28.250000,25.910000,27.620001,92581800,16.437550,0.0,0
2,PETR4.SA,27.799999,28.160000,26.690001,27.070000,85179500,16.110224,0.0,0
3,PETR4.SA,26.580000,27.549999,26.389999,26.990000,57874700,16.062614,0.0,0
4,PETR4.SA,26.980000,27.379999,26.120001,26.490000,69262300,15.765049,0.0,0


## Transformação das matrizes: Separando os dias como objetos numpy.

tentar criar objetos que contenham as matrizes, os rótulos e a identificação do dia.

O último dia não está incluso no treinamento por não possuir rótulo.

Dimensões:
* Medidas:
  * Dia da semana
  * Dia do ano
  * Abertura
  * Mínimo
  * Máximo
  * Fechamento
  * Fechamento Ajustado
  * Volume
* tickers
* Dias (atual e anteriores)

Há uma variável ajustável para determinar quantos dias serão olhados para trás.

Um dicionário de rótulos, um rótulo para cada ticker, é atrelado a cada matriz.

## Separação entre Treino, Teste e Validação.

É preciso garantir que as matrizes sejam embaralhadas, mas que os rótulos permaneçam atrelados adequadamente.

## Modelo

* Para cada ticker (a partir do dicionário de rótulos):  
    * O modelo é treinado seguindo aquele ticker como rótulo (com uma função)  
    * O modelo é validado  
    * Se o modelo é válido:  
        * A previsão é tentada:  
        * Se o resultado é diferente de zero E a probabilidade é alta:  
                * O deploy é salvo